In [12]:
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re

In [ ]:
data_dir = "../data/aruba/data.csv"
df_aruba = pd.read_csv(
    data_dir,
    delim_whitespace=True,
    names=[
        "date",
        "time",
        "sensor_type",
        "sensor_status",
        "meta",
        "meta_begin_end",
    ],
)

pattern_motion_sensor = r"[M]{1}[0-9]{3,}"
pattern_temperature_sensor = r"[T]{1}[0-9]{3,}"
pattern_door_closure_sensor = r"[D]{1}[0-9]{3,}"

# Create subsets for each sensor type
df_motion_sensor = df_aruba[
    df_aruba["sensor_type"].str.match(pattern_motion_sensor)
]
df_temperature_sensor = df_aruba[
    df_aruba["sensor_type"].str.match(pattern_temperature_sensor)
]
df_door_closure_sensor = df_aruba[
    df_aruba["sensor_type"].str.match(pattern_door_closure_sensor)
]

display(df_temperature_sensor)

/tmp/ipykernel_773/739053850.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_aruba = pd.read_csv(


,date,time,sensor_type,sensor_status,meta,meta_begin_end
2,2010-11-04,00:15:08.984841,T002,21.5,NaN,NaN
3,2010-11-04,00:30:19.185547,T003,21,NaN,NaN
4,2010-11-04,00:30:19.385336,T004,21,NaN,NaN
5,2010-11-04,00:35:22.24587,T005,20.5,NaN,NaN
6,2010-11-04,00:40:25.428962,T005,21,NaN,NaN
...,...,...,...,...,...,...
1719553,2011-06-11,23:42:59.28507,T002,25.5,NaN,NaN
1719554,2011-06-11,23:48:02.888409,T001,23.5,NaN,NaN
1719555,2011-06-11,23:48:02.988798,T002,25,NaN,NaN
1719556,2011-06-11,23:53:06.4292,T002,25.5,NaN,NaN
